In [4]:
""" !pip install pytube
!pip install openai-whisper
!pip install langchain
!pip install openai """

' !pip install pytube\n!pip install openai-whisper\n!pip install langchain\n!pip install openai '

In [2]:
from torch import cuda, torch
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [3]:
""" import langchain.output_parsers as parsers
dir(parsers) """


' import langchain.output_parsers as parsers\ndir(parsers) '

In [ ]:
""" from pytube import YouTube

def download_audio_from_youtube(url):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()
    audio_path = audio_stream.download(filename="audio.mp3")
    return audio_path
 """

In [ ]:
""" import whisper

def transcribe_audio_with_whisper(audio_path):
    model = whisper.load_model("base")  # or "small", "medium", "large" depending on needs
    transcription = model.transcribe(audio_path)
    return transcription['text'] """


In [ ]:
""" from langchain.tools import Tool

def youtube_to_text_tool(url):
    audio_path = download_audio_from_youtube(url)
    transcription = transcribe_audio_with_whisper(audio_path)
    return transcription

youtube_transcription_tool = Tool(
    name="YouTubeTranscriptionTool",
    func=youtube_to_text_tool,
    description="Converts YouTube video audio to text."
) """

In [ ]:
#!pip install python-dotenv

In [10]:
from dotenv import load_dotenv, find_dotenv
import getpass
import os

# Load Environment Variable
_ = load_dotenv(find_dotenv())

# Set the environment variable directly in the script
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
# initialize connection to pinecone (get API key at app.pinecone.io)
#PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") # or "YOUR_API_KEY"

In [12]:
# Access the environment variable with os.environ
#print(os.environ["OPENAI_API_KEY"])


In [ ]:
""" !pip install --upgrade pytube
!pip install yt-dlp
!pip install imageio[ffmpeg] """


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.6/171.6 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 12.8 MB/s eta 0:00:0000:010:01mm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 127.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
   

In [ ]:
# TO DEBUG THE FFMPEG location issue!

""" import os
print(os.environ["PATH"])

import subprocess
subprocess.run(["ffmpeg", "-version"]) """

#!apt-get update && apt-get install -y ffmpeg

/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/root/.local/bin:/usr/local/cuda/bin


In [ ]:
#!which ffmpeg

/usr/bin/ffmpeg


In [ ]:
# using TEXT file that has multiple YOUTUBE video links!

import yt_dlp
from whisper import load_model

# Default configurations for easy modification
VIDEO_LINKS_FILE = "video_links.txt"  # Text file with newline-separated YouTube links
AUDIO_OUTPUT_TEMPLATE = "audio_{}.mp3"  # Template for audio filenames with .mp3 extension
WHISPER_MODEL = "turbo"  # Use Whisper turbo for faster processing, if available

# Load YouTube links from file with newline separation
def load_video_links(filename=VIDEO_LINKS_FILE):
    with open(filename, 'r') as file:
        video_links = [link.strip() for link in file.readlines() if link.strip()]
    return video_links

# Download audio from a YouTube video
def download_audio(url, output_name):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }
        ],
        'outtmpl': output_name,  # Ensure the filename includes .mp3 extension
        'ffmpeg_location': '/usr/bin/ffmpeg'  # Update for Paperspace
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return output_name  # Return the filename with .mp3 extension

# Transcribe audio using Whisper turbo model
def transcribe_audio_with_whisper(audio_path, model_name=WHISPER_MODEL):
    model = load_model(model_name)  # Use turbo model for faster processing
    result = model.transcribe(audio_path)
    return result["text"]

# Process each video link
def process_videos(filename=VIDEO_LINKS_FILE):
    video_links = load_video_links(filename)
    all_transcriptions = []
    for i, video_url in enumerate(video_links):
        print(f"Processing video {i+1}/{len(video_links)}: {video_url}")
        audio_path = download_audio(video_url, output_name=AUDIO_OUTPUT_TEMPLATE.format(i))  # Use .mp3 extension
        transcription = transcribe_audio_with_whisper(audio_path)
        all_transcriptions.append(transcription)
        print(f"Completed transcription for video {i+1}")
    return all_transcriptions

# Run the process
all_transcriptions = process_videos()


Processing video 1/1: https://www.youtube.com/watch?v=kgldqMlCh78
[youtube] Extracting URL: https://www.youtube.com/watch?v=kgldqMlCh78
[youtube] kgldqMlCh78: Downloading webpage
[youtube] kgldqMlCh78: Downloading ios player API JSON
[youtube] kgldqMlCh78: Downloading mweb player API JSON
[youtube] kgldqMlCh78: Downloading m3u8 information
[info] kgldqMlCh78: Downloading 1 format(s): 251
[download] audio_0.mp3 has already been downloaded
[download] 100% of    8.07MiB
[ExtractAudio] Not converting audio audio_0.mp3; file is already in target format mp3
Completed transcription for video 1


In [20]:
# Default configuration for consolidated transcription output
CONSOLIDATED_TRANSCRIPTION_FILE = "consolidated_transcription.html"  # Name for the HTML file

# Save consolidated transcription to a single HTML file
def save_consolidated_transcription(transcriptions=all_transcriptions):
    with open(CONSOLIDATED_TRANSCRIPTION_FILE, 'w', encoding='utf-8') as file:
        file.write("<html><body><h1>Consolidated Transcription</h1><ol>")
        for i, transcription in enumerate(transcriptions):
            file.write(f"<h2>Video {i+1}</h2>")
            for line in transcription.splitlines():
                file.write(f"<li>{line}</li>")
        file.write("</ol></body></html>")
    print(f"Transcription saved as {CONSOLIDATED_TRANSCRIPTION_FILE}")

# Save all transcriptions to the consolidated HTML file
save_consolidated_transcription()


Transcription saved as consolidated_transcription.html


In [ ]:
""" # to run it on PAPERSPACE! Youtube link IN-LINE!

import yt_dlp

def download_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }
        ],
        'outtmpl': 'audio_accent_1hr',
        'ffmpeg_location': '/usr/bin/ffmpeg'  # Updated path to ffmpeg on Paperspace
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return 'audio_accent_1hr.mp3'

# Assuming transcribe_audio_with_whisper is a function that takes an audio path and transcribes it with Whisper
video_url = "https://www.youtube.com/watch?v=rhiYfBctLOI"
audio_path = download_audio(video_url)
transcription = transcribe_audio_with_whisper(audio_path)
#print(transcription) """

[youtube] Extracting URL: https://www.youtube.com/watch?v=rhiYfBctLOI
[youtube] rhiYfBctLOI: Downloading webpage
[youtube] rhiYfBctLOI: Downloading ios player API JSON
[youtube] rhiYfBctLOI: Downloading mweb player API JSON
[youtube] rhiYfBctLOI: Downloading player 4e23410d
[youtube] rhiYfBctLOI: Downloading m3u8 information
[info] rhiYfBctLOI: Downloading 1 format(s): 251
[download] Destination: audio_accent_1hr
[download] 100% of   51.43MiB in 00:00:01 at 30.88MiB/s    
[ExtractAudio] Destination: audio_accent_1hr.mp3
Deleting original file audio_accent_1hr (pass -k to keep)


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 179MiB/s]


In [ ]:
""" # Save transcription to HTML
with open('transcription.html', 'w', encoding='utf-8') as file:
    file.write("<html><body><h1>Transcription</h1><ol>")
    for line in transcription.splitlines():  # Split by lines if it's a long string
        file.write(f"<li>{line}</li>")
    file.write("</ol></body></html>")

print("Transcription saved as HTML.") """

Transcription saved as HTML.


In [ ]:
#!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: marshmallow
    Found existing installation: marshmallow 2.21.0
    Uninstalling marshmallow-2.21.0:
      Successfully uninstalled marshmallow-2.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
gradient 2.0.6 requires marshmallow<3.0, but you have marshmallow 3.23.0 which is incompatible.


In [24]:
# SEGMENT and VECTORIZE the Transcription with Langchain

# Join all transcriptions into a single string (Conversion of List to String)
consolidated_text = "\n\n".join(all_transcriptions)

from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the OpenAI embedding model 
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=400)
segments = text_splitter.split_text(consolidated_text)

# Vectorize each segment and store in a list with segment metadata
vectorized_segments = []
embeddings = embedding_model.embed_documents(segments)  # Generate embeddings for all segments

for i, (segment, vector) in enumerate(zip(segments, embeddings)):
    vectorized_segments.append({
        "id": f"segment_{i}",
        "text": segment,
        "embedding": vector
    })

print("Vectorization complete. Ready for storage in a vector database.")

Vectorization complete. Ready for storage in a vector database.


In [25]:
#!pip install pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 23.3 MB/s eta 0:00:00


In [29]:
from pinecone import Pinecone
import os
from getpass import getpass

# Securely fetch OpenAI and Pinecone API keys                    
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY") or getpass("Enter Pinecone API key: ")
#os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass("Enter OpenAI API key: ")

In [27]:
#print(os.environ["PINECONE_API_KEY"])

In [30]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

In [31]:
# EMBEDDINGS OpenAI-ada-002

import os
import time
from getpass import getpass
from tqdm.auto import tqdm
from langchain.embeddings import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec

# Securely fetch OpenAI and Pinecone API keys
#os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass("Enter OpenAI API key: ")
#os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY") or getpass("Enter Pinecone API key: ")
#os.environ["PINECONE_ENV"] = os.getenv("PINECONE_ENV") or "us-west1-gcp"  # Example environment

# Initialize OpenAI embeddings model
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=os.environ["OPENAI_API_KEY"])

# Initialize Pinecone client with the updated API
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
#spec = ServerlessSpec(cloud="aws", region="us-west-1")  # Specify Pinecone environment
index_name = "transcription-diwali-qa-index"

# Check if the index exists; create if it doesn’t
if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embedding dimension
        metric="dotproduct",
        spec=spec
    )
    # Wait for the index to be ready
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

# Connect to the index for upsertion
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [32]:
# Upsert embeddings to Pinecone in batches
batch_size = 50
for i in tqdm(range(0, len(vectorized_segments), batch_size)):
    batch = vectorized_segments[i:i + batch_size]
    ids = [segment["id"] for segment in batch]
    embeds = [segment["embedding"] for segment in batch]
    metadatas = [{"text": segment["text"]} for segment in batch]
    index.upsert(vectors=zip(ids, embeds, metadatas))

print("All segments upserted into Pinecone.")

  0%|          | 0/1 [00:00<?, ?it/s]

All segments upserted into Pinecone.


In [33]:
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10}

In [34]:
# EMBED user QUERY and retrieve RELEVANT SEGMENTS (K-5)
# generate a SUMMARY at first # NEED TO relook at this requirement, as it is only appending the segments. or use GPT4>

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Step 1: Embed User Query and Retrieve Relevant Segments
def retrieve_relevant_segments(query):
    query_embedding = embedding_model.embed_query(query)
    results = index.query(vector=query_embedding, top_k=5, include_metadata=True)
    segments = [match['metadata']['text'] for match in results['matches']]
    return segments


# Step 2: Generate Summary from Retrieved Segments
def generate_summary(segments):
    # Concatenate segments to prepare for summarization
    context = " ".join(segments)
    summary_prompt = PromptTemplate(input_variables=["context"], template="Summarize the following: {context}")
    summary = summary_prompt.format(context=context)
    return summary

# Sample user query
user_query = "Tell me about this subject in simple terms."
segments = retrieve_relevant_segments(user_query)
summary = generate_summary(segments)

print("Summary of Subject:", summary)

Summary of Subject: Summarize the following: Time now for our Entrenews segment and today we're going to learn a little more about the Festival of Lights. Diwali is celebrated in India and is starting to gain popularity around the world. And to talk more about this, let's bring in our very own Delano D'Souza. Delano, let's start at the beginning. What exactly is Diwali? Hi, Jeannie. Good to see you. Well, Diwali is a festival that celebrates essentially the triumph of good over evil, knowledge over ignorance. It's a holiday that's popular amongst Hindus over in India, but is celebrated across religious lines. So Christians, Muslims, Buddhists, Sikhs, everyone celebrates Diwali. But like everything in India, there is a divide between the north and the south over how it is celebrated or even what it is called. But what is common in India is you have a lot of these little lights called diyas that are lit and they're pretty much everywhere. Now, there's also this belief that if you gamble 

In [35]:
# AGENT setup for QA interaction:

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

# Initialize memory to track conversation context
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize the Chat model (e.g., GPT-3.5)
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"])

# Wrap Pinecone index with LangChain's Pinecone vector store
vector_store = Pinecone(index=index, embedding=embedding_model, text_key="text")  # Specify "text" as the key for document content

# Create Conversational Retrieval Chain using the wrapped vector store and memory
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=chat_model,
    retriever=vector_store.as_retriever(),  # Use the vector store as the retriever
    memory=memory
)

# Define a function to handle the initial interaction
def initial_interaction(user_query):
    # Step 1: Retrieve and summarize
    segments = retrieve_relevant_segments(user_query)
    summary = generate_summary(segments)
    
    # Step 2: Present the summary to the user and prompt for follow-ups
    print("Summary of Subject:", summary)
    follow_up_query = input("Do you have any further questions on this topic? ")
    
    # Step 3: Run the follow-up query through the agent
    response = qa_chain({"question": follow_up_query})
    print("Agent's Response:", response['answer'])

# Example call to start the interaction
user_query = "Tell me about this subject in simple terms."
initial_interaction(user_query)


/tmp/ipykernel_40/1154057344.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipykernel_40/1154057344.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"])
/tmp/ipykernel_40/1154057344.py:14: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip 

Summary of Subject: Summarize the following: Time now for our Entrenews segment and today we're going to learn a little more about the Festival of Lights. Diwali is celebrated in India and is starting to gain popularity around the world. And to talk more about this, let's bring in our very own Delano D'Souza. Delano, let's start at the beginning. What exactly is Diwali? Hi, Jeannie. Good to see you. Well, Diwali is a festival that celebrates essentially the triumph of good over evil, knowledge over ignorance. It's a holiday that's popular amongst Hindus over in India, but is celebrated across religious lines. So Christians, Muslims, Buddhists, Sikhs, everyone celebrates Diwali. But like everything in India, there is a divide between the north and the south over how it is celebrated or even what it is called. But what is common in India is you have a lot of these little lights called diyas that are lit and they're pretty much everywhere. Now, there's also this belief that if you gamble 

/tmp/ipykernel_40/1154057344.py:34: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": follow_up_query})


Agent's Response: Diwali is celebrated outside India by the Indian diaspora in various countries where there is a significant Hindu population. The celebrations usually include lighting diyas (small oil lamps), decorating homes with colorful rangoli designs, exchanging gifts, wearing new clothes, enjoying festive meals, and setting off fireworks. The cultural significance of Diwali is shared through community events, religious ceremonies, and cultural performances. In recent years, some places have also recognized Diwali as an official holiday, allowing for wider participation and acknowledgment of the festival's importance.


In [38]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
import json

# Define a simple response schema with only the main answer
response_schema = [
    ResponseSchema(name="answer", description="Main response text in a complete paragraph")
]

# Initialize the StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

# Define the prompt template with stricter JSON formatting instructions
response_template = PromptTemplate(
    template=(
        "Answer the question below in a complete paragraph and respond in *strict JSON* format:\n\n"
        "{format_instructions}\n\n"
        "Question: {question}"
    ),
    input_variables=["question"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Interactive chat function with improved parsing logic
def interactive_chat():
    print("Welcome to the Q&A Chatbot! Ask me anything.")
    while True:
        user_query = input("You: ")
        if user_query.lower() == "no":
            print("Agent: Thank you for using the Q&A chatbot. Goodbye!")
            break
        
        # Generate response with format instructions
        formatted_prompt = response_template.format(question=user_query)
        response = qa_chain({"question": formatted_prompt})
        
        # Check for JSON format; fallback to raw response if JSON parsing fails
        try:
            parsed_response = output_parser.parse(response['answer'])
            print("Agent's Response:\n", parsed_response['answer'])
        except Exception:
            print("Error parsing response, displaying raw answer instead.")
            print("Raw Response:\n", response['answer'])
        
        print("\nAgent: Do you have more questions?")

# Start the interactive chat
interactive_chat()


Welcome to the Q&A Chatbot! Ask me anything.
Error parsing response, displaying raw answer instead.
Raw Response:
 The U.S. state that recently signed a bill recognizing Diwali as a state holiday is Pennsylvania.

Agent: Do you have more questions?
Error parsing response, displaying raw answer instead.
Raw Response:
 Diwali is typically celebrated in the autumn, usually in October or November, depending on the lunar calendar.

Agent: Do you have more questions?
Error parsing response, displaying raw answer instead.
Raw Response:
 Diwali is typically celebrated in the autumn, usually between mid-October and mid-November.

Agent: Do you have more questions?
Error parsing response, displaying raw answer instead.
Raw Response:
 Diwali is primarily celebrated by Hindus in India, but it is also celebrated by people of other faiths such as Christians, Muslims, Buddhists, and Sikhs. It is a festival that is celebrated across religious lines.

Agent: Do you have more questions?
Error parsing re

In [21]:
# TAMIL TRANSCRIPTION GENERATION 


""" from whisper import load_model

# Load the Whisper model
model1 = load_model("large")  # or use "small" or "base" for faster processing with lower accuracy


# Transcribe the Tamil audio file
result = model1.transcribe("audio_tamil.mp3", language="ta")
transcription = result["text"]

print(transcription) """

100%|█████████████████████████████████████| 2.88G/2.88G [00:53<00:00, 57.4MiB/s]


 இப்போது புதிய அறிமுகம் அச்சி ஹோட்டல் சாம்பார் 50 கிராம் ரும்பாய் 20 மட்டுமே சென்னை பெருநகர் மற்றும் புரனகரில் சுமார் ஒரு மணி நேரமாக வெளித்து வாங்கும் கணமழை ராயப் பேட்டை, தியாகராய நகர், எழும்பூர், அண்ணாசாலை, அண்ணா நகர் meantime anchor உள்ளிட்ட பகுதிகளில் இடி மின்னவுடன் கொட்டி தீர்க்கும் கணமழை சென்னையில் மேற்கு அண்ணா நகர் பகுதியில் ஒரு மணி நேரத்துயில் 90 cómbing kindergarten அளவிருக்கு கணமழை பதிபு கொளத்தூர் பெரம்பூர் அம்பத்தூர் அமைந்தகரை ஆகிய இடங்கள் அதிக மழை பொழிவு எதிர்க்கொண்டதாக வானிலை மயம் தகவன் பசும்புன் முத்துராமலிங்க தேவரி 117. ஜெயன்தி மற்றும் 62. குருபுஜகி உட்டி முதல்மைச்சர் முக்கு ச்டாலின் மரியாது பசும்புன்னில் உள்ள தேவரி நினிவிடத்தில் மளர்த்தூபி சிலைக்கு மாலை நிவித்த மரியாதை சொல்றினார் பசும்புன்னில் தேவர் வாழ்ந்த இளம் புதுப்பிக்கப்பட்டு 100 ஆண்டு அலங்கाர வளைவு நினைவிடத்தில் அணயா விளக்கு அமைப்பு தேவர் ஜெயன்தியின் போது பசும்புன்னில் கூட நரிசலை தவிர்க தேவர் அரங்கம் திருக்கப்பட்டுள்ளதாக முதலாமைச்சர் முக்கஸ் டாலின் பேட்டி முதலாமைச்சர் முக்கஸ் டாலின் பேட்டி முதலாமைச்சர் முக்கஸ் டாலி

In [39]:
""" # to run it on CPU!

import yt_dlp

def download_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }
        ],
        'outtmpl': 'audio.mp3',
        'ffmpeg_location': 'C:/Users/KK/AppData/Local/ffmpeg-essentials_build/bin'  # Explicit path to ffmpeg
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return 'audio.mp3'

# Assuming transcribe_audio_with_whisper is a function that takes an audio path and transcribes it with Whisper
video_url = "https://www.youtube.com/watch?v=RRk9QlCjul0"
audio_path = download_audio(video_url)
transcription = transcribe_audio_with_whisper(audio_path)
print(transcription) """


' # to run it on CPU!\n\nimport yt_dlp\n\ndef download_audio(url):\n    ydl_opts = {\n        \'format\': \'bestaudio/best\',\n        \'postprocessors\': [\n            {\n                \'key\': \'FFmpegExtractAudio\',\n                \'preferredcodec\': \'mp3\',\n                \'preferredquality\': \'192\',\n            }\n        ],\n        \'outtmpl\': \'audio.mp3\',\n        \'ffmpeg_location\': \'C:/Users/KK/AppData/Local/ffmpeg-essentials_build/bin\'  # Explicit path to ffmpeg\n    }\n    with yt_dlp.YoutubeDL(ydl_opts) as ydl:\n        ydl.download([url])\n    return \'audio.mp3\'\n\n# Assuming transcribe_audio_with_whisper is a function that takes an audio path and transcribes it with Whisper\nvideo_url = "https://www.youtube.com/watch?v=RRk9QlCjul0"\naudio_path = download_audio(video_url)\ntranscription = transcribe_audio_with_whisper(audio_path)\nprint(transcription) '